In [1]:
import pandas as pd
df = pd.read_csv('tendency_to_obesity.csv')
X = df.drop('NObeyesdad',axis=1)
y = df.NObeyesdad
special_columns = ['FCVC', 'NCP', 'CH2O', 'FAF', 'TUE','Age']
# видимо, данные при чтении(записи) были плохо записаны, найденная мной информация о датасете говорит, 
# что должны быть целые числа,
# причем данные 'не количественные', к примеру: FAF  - Frequency of physical activity -> 0 ==do not have,1 ==1 0r 2 days
# однозначно стоит разбить на dummies(все, кроме Age)
for column in special_columns:
    X[column] = X[column].astype('uint8')

In [2]:
def getting_and_changeing_columns_to_dummieses(df, n):
    """If count of unique values is smaller than n , we create dummies"""
    columns = df.columns
    for column in columns:
        if len(df[column].unique()) < n:
        
            df = pd.concat([df,pd.get_dummies(X[column],prefix=column)],axis=1)
            df.drop(column,axis=1,inplace=True)
    return df

In [3]:
X = getting_and_changeing_columns_to_dummieses(X,10)

In [4]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

In [5]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(X = X_train)
X_train = ss.transform(X_train)
ss = StandardScaler().fit(X = X_test)
X_test = ss.transform(X_test)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=115.95959595959596,solver='newton-cg',random_state=42)
dt = DecisionTreeClassifier(max_depth=13, random_state=42)


In [7]:
from sklearn.ensemble import StackingClassifier
sc1 = StackingClassifier(final_estimator=dt,n_jobs=-1,estimators = [('lr',lr),('dt',dt)],passthrough=True)
sc1.fit(X_train,y_train)
print(classification_report(y_test,sc1.predict(X_test)))

                     precision    recall  f1-score   support

Insufficient_Weight       0.89      0.96      0.92        56
      Normal_Weight       0.95      0.87      0.91        62
     Obesity_Type_I       0.99      0.95      0.97        78
    Obesity_Type_II       0.95      1.00      0.97        58
   Obesity_Type_III       1.00      1.00      1.00        63
 Overweight_Level_I       0.98      0.95      0.96        56
Overweight_Level_II       0.94      0.98      0.96        50

           accuracy                           0.96       423
          macro avg       0.96      0.96      0.96       423
       weighted avg       0.96      0.96      0.96       423

